In [8]:
!pip install -q tf-agents==0.19.0


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [19]:
import functools
import os

import tensorflow as tf
from tf_agents.bandits.agents import dropout_thompson_sampling_agent as dropout_ts_agent
from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.bandits.agents import linear_thompson_sampling_agent as lin_ts_agent
from tf_agents.bandits.agents import neural_epsilon_greedy_agent as eps_greedy_agent
from tf_agents.bandits.agents.examples.v2 import trainer
from tf_agents.bandits.environments import environment_utilities
from tf_agents.bandits.environments import movielens_per_arm_py_environment
from tf_agents.bandits.environments import movielens_py_environment
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.bandits.networks import global_and_arm_feature_network
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network

In [20]:
# Constants
BATCH_SIZE = 8
TRAINING_LOOPS = 20000
STEPS_PER_LOOP = 2

RANK_K = 20
NUM_ACTIONS = 20

AGENT_ALPHA = 10.0
EPSILON = 0.05
LAYERS = (50, 50, 50)
LR = 0.005
DROPOUT_RATE = 0.2


In [22]:

def run_bandit_training(root_dir, data_path, agent_name="LinUCB", per_arm=False):
    tf.compat.v1.enable_v2_behavior()

    if not data_path:
        raise ValueError('Please specify the location of the MovieLens data file.')

    # Environment
    if per_arm:
        env = movielens_per_arm_py_environment.MovieLensPerArmPyEnvironment(
            data_path, RANK_K, BATCH_SIZE, num_actions=NUM_ACTIONS, csv_delimiter='\t')
    else:
        env = movielens_py_environment.MovieLensPyEnvironment(
            data_path, RANK_K, BATCH_SIZE, num_movies=NUM_ACTIONS, csv_delimiter='\t')

    environment = tf_py_environment.TFPyEnvironment(env)

    # Optimal reward/action functions
    optimal_reward_fn = functools.partial(
        environment_utilities.compute_optimal_reward_with_movielens_environment,
        environment=environment)
    optimal_action_fn = functools.partial(
        environment_utilities.compute_optimal_action_with_movielens_environment,
        environment=environment)

    # Agent selection
    if agent_name == 'LinUCB':
        agent = lin_ucb_agent.LinearUCBAgent(
            time_step_spec=environment.time_step_spec(),
            action_spec=environment.action_spec(),
            tikhonov_weight=0.001,
            alpha=AGENT_ALPHA,
            dtype=tf.float32,
            accepts_per_arm_features=per_arm,
        )
    elif agent_name == 'LinTS':
        agent = lin_ts_agent.LinearThompsonSamplingAgent(
            time_step_spec=environment.time_step_spec(),
            action_spec=environment.action_spec(),
            dtype=tf.float32,
            accepts_per_arm_features=per_arm,
        )
    elif agent_name == 'epsGreedy':
        if per_arm:
            network = global_and_arm_feature_network.create_feed_forward_dot_product_network(
                environment.time_step_spec().observation,
                global_layers=LAYERS,
                arm_layers=LAYERS,
            )
        else:
            network = q_network.QNetwork(
                input_tensor_spec=environment.time_step_spec().observation,
                action_spec=environment.action_spec(),
                fc_layer_params=LAYERS,
            )
        agent = eps_greedy_agent.NeuralEpsilonGreedyAgent(
            time_step_spec=environment.time_step_spec(),
            action_spec=environment.action_spec(),
            reward_network=network,
            optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=LR),
            epsilon=EPSILON,
            emit_policy_info='predicted_rewards_mean',
            info_fields_to_inherit_from_greedy=['predicted_rewards_mean'],
        )
    elif agent_name == 'DropoutTS':
        train_step_counter = tf.compat.v1.train.get_or_create_global_step()

        def dropout_fn():
            return tf.math.maximum(
                tf.math.reciprocal_no_nan(1.01 + tf.cast(train_step_counter, tf.float32)),
                0.0003,
            )

        agent = dropout_ts_agent.DropoutThompsonSamplingAgent(
            time_step_spec=environment.time_step_spec(),
            action_spec=environment.action_spec(),
            dropout_rate=dropout_fn,
            network_layers=LAYERS,
            optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=LR),
        )
    else:
        raise ValueError(f"Unknown agent: {agent_name}")

    # Metrics
    regret_metric = tf_bandit_metrics.RegretMetric(optimal_reward_fn)
    suboptimal_arms_metric = tf_bandit_metrics.SuboptimalArmsMetric(optimal_action_fn)
    
    # Train the agent
    trainer.train(
        root_dir=root_dir,
        agent=agent,
        environment=environment,
        training_loops=TRAINING_LOOPS,
        steps_per_loop=STEPS_PER_LOOP,
        additional_metrics=[regret_metric, suboptimal_arms_metric],
    )

In [ ]:
    run_bandit_training(
    root_dir='/tmp/movielens_bandit_output',
    data_path='u.data',
    agent_name='LinUCB',  # Options: LinUCB, LinTS, epsGreedy, DropoutTS
    per_arm=False
)


W0407 01:30:19.293223 139877721212096 polymorphic_function.py:156] 5 out of the last 5 calls to <function TFStepMetric._update_state at 0x7f37025d32e0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


W0407 01:30:19.566129 139877721212096 polymorphic_function.py:156] 6 out of the last 6 calls to <function TFStepMetric._update_state at 0x7f3702318790> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
W0407 01:30:21.213129 139877721212096 function_type.py:509] `0/step_type` is not a valid tf.function parameter name. Sanitizing to `arg_0_step_type`.
W0407 01:30:21.214294 139877721212096 function_type.py:509] `0/reward` is not a valid tf.function parameter name. Sanitizing to `ar

INFO:tensorflow:Assets written to: /tmp/movielens_bandit_output/policy_16/assets


/opt/conda/lib/python3.10/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
I0407 01:30:22.794469 139877721212096 builder_impl.py:801] Assets written to: /tmp/movielens_bandit_output/policy_16/assets


INFO:tensorflow:Assets written to: /tmp/movielens_bandit_output/policy_1616/assets


...

/opt/conda/lib/python3.10/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
I0407 02:48:56.022187 139877721212096 builder_impl.py:801] Assets written to: /tmp/movielens_bandit_output/policy_99216/assets


INFO:tensorflow:Assets written to: /tmp/movielens_bandit_output/policy_100816/assets


/opt/conda/lib/python3.10/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
I0407 02:50:12.061190 139877721212096 builder_impl.py:801] Assets written to: /tmp/movielens_bandit_output/policy_100816/assets


INFO:tensorflow:Assets written to: /tmp/movielens_bandit_output/policy_102416/assets


/opt/conda/lib/python3.10/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
I0407 02:51:27.884116 139877721212096 builder_impl.py:801] Assets written to: /tmp/movielens_bandit_output/policy_102416/assets


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/movielens_bandit_output
